In [4]:
!pip -q install rasterio geopandas shapely fiona pyproj rtree ipywidgets

import os, glob, zipfile, re
import numpy as np
import pandas as pd
import rasterio
from rasterio.windows import from_bounds
from rasterio.mask import mask
import geopandas as gpd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from google.colab import files


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.2 MB/s eta 0:00:00


In [5]:
uploaded = files.upload()  # upload your shapefile.zip
zip_name = next(iter(uploaded.keys()))

SHAPE_DIR = "/content/aoi_shape"
os.makedirs(SHAPE_DIR, exist_ok=True)

with zipfile.ZipFile(zip_name, "r") as z:
    z.extractall(SHAPE_DIR)

# load shapefile (find .shp)
shps = glob.glob(os.path.join(SHAPE_DIR, "**", "*.shp"), recursive=True)
print("Found shp:", shps)

aoi = gpd.read_file(shps[0])
aoi = aoi.to_crs("EPSG:4326")  # OISST যদি lon/lat হয়
aoi_union = aoi.unary_union    # single geometry
print(aoi.head())


Saving Bay of Bengal.zip to Bay of Bengal (2).zip
Found shp: ['/content/aoi_shape/Bay of Bengal/iho.shp']
            name  id  longitude   latitude     min_x     min_y     max_x  \
0  Bay of Bengal  43  87.454425  13.173222  78.89822  5.733971  95.04884   

       max_y       area  mrgid  \
0  24.377669  2207565.0   4273   

                                            geometry  
0  POLYGON ((79.69563 9.07902, 79.67974 9.07284, ...  


/tmp/ipython-input-3257259435.py:16: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  aoi_union = aoi.unary_union    # single geometry


In [6]:
import os, zipfile, glob
from google.colab import files

# 1) Upload OISST zip from your computer
uploaded = files.upload()   # select your OISST.zip
zip_name = next(iter(uploaded.keys()))
print("Uploaded:", zip_name)

# 2) Unzip location
BASE_DIR = "/content/OISST_TIF"
os.makedirs(BASE_DIR, exist_ok=True)

# 3) Unzip
with zipfile.ZipFile(zip_name, "r") as z:
    z.extractall(BASE_DIR)

print("Unzipped to:", BASE_DIR)

# 4) Find tif files anywhere inside
tifs = glob.glob(os.path.join(BASE_DIR, "**", "*.tif"), recursive=True)
print("Total tif found:", len(tifs))
print("Example:", tifs[:5])


Saving OISST.zip to OISST.zip
Uploaded: OISST.zip
Unzipped to: /content/OISST_TIF
Total tif found: 524
Example: ['/content/OISST_TIF/OISST/2003 July.tif', '/content/OISST_TIF/OISST/2009 Dec.tif', '/content/OISST_TIF/OISST/1998 Mar.tif', '/content/OISST_TIF/OISST/2001 Mar.tif', '/content/OISST_TIF/OISST/1988 July.tif']


In [7]:
def aoi_area_weighted_mean(tif_path, aoi_geom, is_kelvin=False):
    with rasterio.open(tif_path) as src:
        # mask to AOI (crop True)
        out_img, out_transform = mask(src, [aoi_geom], crop=True, filled=True, nodata=np.nan)
        arr = out_img[0].astype("float64")

        if src.nodata is not None:
            arr[arr == src.nodata] = np.nan

        if is_kelvin:
            arr = arr - 273.15

        if np.all(~np.isfinite(arr)):
            return np.nan

        # lat of pixel centers from transform
        nrows, ncols = arr.shape
        rows = np.arange(nrows)
        # y = f + (row+0.5)*e
        ys = out_transform.f + (rows + 0.5) * out_transform.e
        w = np.cos(np.deg2rad(ys))[:, None]

        num = np.nansum(arr * w)
        den = np.nansum(w * np.isfinite(arr))
        return num / den if den != 0 else np.nan


In [8]:
from rasterio.mask import mask

def trend_map_for_month(selected_month_num, pairs, aoi_geom, is_kelvin=False, min_count=24):
    # collect files for that month
    month_files = [(y,m,fp) for (y,m,fp) in pairs if m == selected_month_num]
    month_files.sort()

    dates = pd.to_datetime([f"{y}-{m:02d}-01" for (y,m,fp) in month_files])
    x = (dates - dates.min()).days.values / 365.25
    x = x.astype("float64")

    stack = []
    ref_transform = None

    for (y,m,fp) in month_files:
        with rasterio.open(fp) as src:
            # ✅ filled=False (masked array) → then convert masked to NaN
            out_img, out_transform = mask(src, [aoi_geom], crop=True, filled=False)

            arr = out_img[0].astype("float64")
            arr[out_img[0].mask] = np.nan   # mask অংশ NaN

            if src.nodata is not None:
                arr[arr == src.nodata] = np.nan

            if is_kelvin:
                arr = arr - 273.15

            if ref_transform is None:
                ref_transform = out_transform

        stack.append(arr)

    stack = np.stack(stack, axis=0)  # (T,H,W)
    T,H,W = stack.shape

    valid = np.isfinite(stack)
    count = valid.sum(axis=0)
    enough = count >= min_count

    x3 = x[:,None,None]

    # safe mean via sum/count (warning-free)
    x_sum = np.sum(np.where(valid, x3, 0.0), axis=0)
    y_sum = np.sum(np.where(valid, stack, 0.0), axis=0)

    x_mean = np.full((H,W), np.nan)
    y_mean = np.full((H,W), np.nan)
    anym = count > 0
    x_mean[anym] = x_sum[anym] / count[anym]
    y_mean[anym] = y_sum[anym] / count[anym]

    dx = x3 - x_mean
    dy = stack - y_mean
    cov = np.sum(np.where(valid, dx*dy, 0.0), axis=0)
    var = np.sum(np.where(valid, dx*dx, 0.0), axis=0)

    slope_per_year = np.full((H,W), np.nan)
    ok = enough & (var > 0)
    slope_per_year[ok] = cov[ok] / var[ok]

    trend_map = (slope_per_year * 10.0).astype("float32")  # °C/decade
    return trend_map, ref_transform


In [6]:
# ----- Install (run once) -----
!pip -q install rasterio geopandas shapely fiona pyproj rtree ipywidgets

import os, glob, zipfile, re
import numpy as np
import pandas as pd
import rasterio
from rasterio.windows import from_bounds
from rasterio.mask import mask
import geopandas as gpd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from google.colab import files

# -----------------------------
# 1) Upload + unzip shapefile zip (AOI)
# -----------------------------
uploaded = files.upload()  # upload your shapefile.zip
zip_name = next(iter(uploaded.keys()))

SHAPE_DIR = "/content/aoi_shape"
os.makedirs(SHAPE_DIR, exist_ok=True)

with zipfile.ZipFile(zip_name, "r") as z:
    z.extractall(SHAPE_DIR)

shps = glob.glob(os.path.join(SHAPE_DIR, "**", "*.shp"), recursive=True)
if len(shps) == 0:
    raise ValueError("No .shp found inside the uploaded zip.")

aoi = gpd.read_file(shps[0])

# IMPORTANT: OISST GeoTIFF সাধারণত lon/lat হয়
aoi = aoi.to_crs("EPSG:4326")
aoi_union = aoi.unary_union

print("Loaded AOI shp:", shps[0])
print(aoi.head())

# -----------------------------
# 2) CONFIG
# -----------------------------
OISST_DIR = "/content/OISST_TIF/OISST"   # <-- আপনার actual folder
OISST_IS_KELVIN = False

# -----------------------------
# 3) Index OISST tifs -> (year,month,fp)
# filenames like: 1982_Jan.tif
# -----------------------------
MONTHS = ["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]
MONTH_NUM = {m:i+1 for i,m in enumerate(MONTHS)}

def parse_year_month(fp):
    name = os.path.basename(fp)
    y = re.search(r"(19|20)\d{2}", name)
    if not y:
        return None
    year = int(y.group(0))

    mon = None
    for mm in MONTHS:
        if mm.lower() in name.lower():
            mon = MONTH_NUM[mm]
            break
    if mon is None:
        return None
    return year, mon

tifs = glob.glob(os.path.join(OISST_DIR, "**", "*.tif"), recursive=True)
pairs = []
for fp in tifs:
    ym = parse_year_month(fp)
    if ym:
        pairs.append((ym[0], ym[1], fp))

pairs.sort()
print("Total tif found:", len(tifs))
print("Indexed (year,month,tif):", len(pairs))
print("First 3:", pairs[:3])
print("Last 3:", pairs[-3:])

if len(pairs) == 0:
    raise ValueError("No monthly tifs indexed. Check OISST_DIR or filenames.")

# -----------------------------
# 4) AOI area-weighted mean function (FIXED: filled=False)
# -----------------------------
def aoi_area_weighted_mean(tif_path, aoi_geom, is_kelvin=False):
    with rasterio.open(tif_path) as src:
        # filled=False → masked array (safe for integer rasters)
        out_img, out_transform = mask(src, [aoi_geom], crop=True, filled=False)

        arr = out_img[0].astype("float64")
        arr[out_img[0].mask] = np.nan

        # nodata -> NaN
        if src.nodata is not None:
            arr[arr == src.nodata] = np.nan

        if is_kelvin:
            arr = arr - 273.15

        if np.all(~np.isfinite(arr)):
            return np.nan

        nrows, ncols = arr.shape
        rows = np.arange(nrows)
        ys = out_transform.f + (rows + 0.5) * out_transform.e
        w = np.cos(np.deg2rad(ys))[:, None]

        num = np.nansum(arr * w)
        den = np.nansum(w * np.isfinite(arr))
        return num / den if den != 0 else np.nan

# -----------------------------
# 5) Build monthly time series table (all months)
# -----------------------------
rows = []
for year, mon, fp in pairs:
    mean_val = aoi_area_weighted_mean(fp, aoi_union, is_kelvin=OISST_IS_KELVIN)
    rows.append((year, mon, pd.Timestamp(year=year, month=mon, day=1), mean_val))

df = pd.DataFrame(rows, columns=["year","month","date","sst"])
print(df.head(), "\nRows:", len(df))

# -----------------------------
# 6) Month-specific pixel-wise trend map (FIXED: filled=False + shape guard)
# -----------------------------
def trend_map_for_month(selected_month_num, pairs, aoi_geom, is_kelvin=False, min_count=24):
    month_files = [(y,m,fp) for (y,m,fp) in pairs if m == selected_month_num]
    month_files.sort()

    dates = pd.to_datetime([f"{y}-{m:02d}-01" for (y,m,fp) in month_files])
    x = (dates - dates.min()).days.values / 365.25
    x = x.astype("float64")

    stack = []
    x_keep = []
    ref_transform = None
    ref_shape = None
    ref_profile = None

    for (y,m,fp), xval in zip(month_files, x):
        with rasterio.open(fp) as src:
            out_img, out_transform = mask(src, [aoi_geom], crop=True, filled=False)
            band = out_img[0]

            arr = band.astype("float64")
            arr[band.mask] = np.nan

            if src.nodata is not None:
                arr[arr == src.nodata] = np.nan
            if is_kelvin:
                arr = arr - 273.15

            # reference shape/transform (first valid)
            if ref_shape is None:
                ref_shape = arr.shape
                ref_transform = out_transform
                ref_profile = src.profile.copy()

        # skip if shape mismatch (rare but can happen)
        if arr.shape != ref_shape:
            continue

        stack.append(arr)
        x_keep.append(xval)

    if len(stack) < min_count:
        raise ValueError(f"Too few valid rasters for this month: {len(stack)} (<{min_count}).")

    stack = np.stack(stack, axis=0)  # (T,H,W)
    x_keep = np.array(x_keep, dtype="float64")

    T, H, W = stack.shape

    valid = np.isfinite(stack)
    count = valid.sum(axis=0)
    enough = count >= min_count

    x3 = x_keep[:, None, None]

    # safe mean via sum/count (no warnings)
    x_sum = np.sum(np.where(valid, x3, 0.0), axis=0)
    y_sum = np.sum(np.where(valid, stack, 0.0), axis=0)

    x_mean = np.full((H,W), np.nan, dtype="float64")
    y_mean = np.full((H,W), np.nan, dtype="float64")
    anym = count > 0
    x_mean[anym] = x_sum[anym] / count[anym]
    y_mean[anym] = y_sum[anym] / count[anym]

    dx = x3 - x_mean
    dy = stack - y_mean

    cov = np.sum(np.where(valid, dx * dy, 0.0), axis=0)
    var = np.sum(np.where(valid, dx * dx, 0.0), axis=0)

    slope_per_year = np.full((H,W), np.nan, dtype="float64")
    ok = enough & (var > 0)
    slope_per_year[ok] = cov[ok] / var[ok]

    trend_map = (slope_per_year * 10.0).astype("float32")  # °C/decade

    return trend_map, ref_transform

# -----------------------------
# 7) Interactive dropdown plot (line + map)
# -----------------------------
month_names = {
    1:"January",2:"February",3:"March",4:"April",5:"May",6:"June",
    7:"July",8:"August",9:"September",10:"October",11:"November",12:"December"
}

dropdown = widgets.Dropdown(
    options=[(month_names[i], i) for i in range(1,13)],
    value=1,
    description="Month:"
)

def plot_month_view(mnum):
    dmon = df[df["month"] == mnum].sort_values("date")

    # trend map
    tmap, t_transform = trend_map_for_month(mnum, pairs, aoi_union, is_kelvin=OISST_IS_KELVIN)

    # extent from transform
    H, W = tmap.shape
    left = t_transform.c
    top  = t_transform.f
    right = left + W * t_transform.a
    bottom = top + H * t_transform.e

    fig, ax = plt.subplots(1, 2, figsize=(13,5))

    # Line
    ax[0].plot(dmon["date"], dmon["sst"], lw=1.5)
    ax[0].set_title(f"{month_names[mnum]} SST (AOI mean) 1982–2025")
    ax[0].set_ylabel("SST (°C)")
    ax[0].grid(True)

    # Map
    im = ax[1].imshow(
        tmap, origin="upper", interpolation="nearest",
        extent=[left, right, bottom, top],
        vmin=-1.0, vmax=1.0
    )
    aoi.boundary.plot(ax=ax[1], color="black", linewidth=1)
    ax[1].set_title(f"{month_names[mnum]} Trend map (°C/decade)")
    ax[1].set_xlabel("Lon"); ax[1].set_ylabel("Lat")
    plt.colorbar(im, ax=ax[1], fraction=0.046, pad=0.04, label="°C/decade")

    plt.tight_layout()
    plt.show()

out = widgets.Output()

def on_change(change):
    if change["name"] == "value":
        out.clear_output(wait=True)
        with out:
            plot_month_view(change["new"])

dropdown.observe(on_change)

display(dropdown, out)

with out:
    plot_month_view(dropdown.value)


Saving Bay of Bengal.zip to Bay of Bengal (1).zip


/tmp/ipython-input-855868960.py:35: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  aoi_union = aoi.unary_union


Loaded AOI shp: /content/aoi_shape/Bay of Bengal/iho.shp
            name  id  longitude   latitude     min_x     min_y     max_x  \
0  Bay of Bengal  43  87.454425  13.173222  78.89822  5.733971  95.04884   

       max_y       area  mrgid  \
0  24.377669  2207565.0   4273   

                                            geometry  
0  POLYGON ((79.69563 9.07902, 79.67974 9.07284, ...  
Total tif found: 524
Indexed (year,month,tif): 523
First 3: [(1982, 1, '/content/OISST_TIF/OISST/1982_Jan.tif'), (1982, 2, '/content/OISST_TIF/OISST/1982_Feb.tif'), (1982, 3, '/content/OISST_TIF/OISST/1982_Mar.tif')]
Last 3: [(2025, 9, '/content/OISST_TIF/OISST/2025 Sep.tif'), (2025, 10, '/content/OISST_TIF/OISST/2025 Oct.tif'), (2025, 11, '/content/OISST_TIF/OISST/2025 Nov.tif')]
   year  month       date        sst
0  1982      1 1982-01-01  26.448879
1  1982      2 1982-02-01  26.940683
2  1982      3 1982-03-01  27.890862
3  1982      4 1982-04-01  29.087626
4  1982      5 1982-05-01  29.560397 
Rows

Dropdown(description='Month:', options=(('January', 1), ('February', 2), ('March', 3), ('April', 4), ('May', 5…

Output()

In [9]:
!pip -q install scipy reportlab

import os, glob, re, zipfile
import numpy as np
import pandas as pd
import rasterio
from rasterio.mask import mask
import geopandas as gpd
import matplotlib.pyplot as plt
from scipy.stats import linregress
from google.colab import files

# =============================
# CONFIG
# =============================
OUT_DIR = "/content/oisst_trend_outputs"
os.makedirs(OUT_DIR, exist_ok=True)

OISST_DIR = "/content/OISST_TIF/OISST"   # OISST monthly tif folder
SHAPE_DIR = "/content/aoi_shape"         # AOI extract folder
OISST_IS_KELVIN = False

# outlier limits for AOI mean SST (line trend + map time-step filter)
SST_MIN, SST_MAX = 24.0, 32.0

# trend map color range (tight for clarity)
VMIN, VMAX = 0.0, 0.6  # °C/decade

month_names = {
    1:"January",2:"February",3:"March",4:"April",5:"May",6:"June",
    7:"July",8:"August",9:"September",10:"October",11:"November",12:"December"
}

MONTHS = ["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]
MONTH_NUM = {m:i+1 for i,m in enumerate(MONTHS)}

# =============================
# Helpers
# =============================
def format_p(p):
    if p is None or (not np.isfinite(p)):
        return "NA"
    return f"{p:.6f}".rstrip("0").rstrip(".")

def parse_year_month(fp):
    name = os.path.basename(fp)
    y = re.search(r"(19|20)\d{2}", name)
    if not y:
        return None
    year = int(y.group(0))
    mon = None
    for mm in MONTHS:
        if mm.lower() in name.lower():
            mon = MONTH_NUM[mm]
            break
    return (year, mon) if mon else None

# =============================
# 1) Upload + extract AOI shapefile zip
# =============================
print("⬆️ Upload AOI shapefile ZIP (contains .shp/.shx/.dbf/.prj)")
uploaded = files.upload()
zip_name = next(iter(uploaded.keys()))
print("Uploaded:", zip_name)

os.makedirs(SHAPE_DIR, exist_ok=True)
with zipfile.ZipFile(zip_name, "r") as z:
    z.extractall(SHAPE_DIR)

shps = glob.glob(os.path.join(SHAPE_DIR, "**", "*.shp"), recursive=True)
print("Found shp:", shps)
if not shps:
    raise FileNotFoundError("No .shp found inside zip. Ensure shapefile components exist.")

aoi = gpd.read_file(shps[0]).to_crs("EPSG:4326")
aoi_union = aoi.unary_union
print("✅ AOI loaded")

# =============================
# 2) Index OISST monthly tifs
# =============================
tifs = glob.glob(os.path.join(OISST_DIR, "**", "*.tif"), recursive=True)
pairs = []
for fp in tifs:
    ym = parse_year_month(fp)
    if ym:
        pairs.append((ym[0], ym[1], fp))
pairs.sort()

print("Total tif found:", len(tifs))
print("Indexed (year,month,tif):", len(pairs))
if not pairs:
    raise FileNotFoundError(f"No valid OISST tifs indexed in {OISST_DIR}. Check folder and filenames.")

# =============================
# 3) AOI area-weighted mean SST
# =============================
def aoi_area_weighted_mean(tif_path, aoi_geom, is_kelvin=False):
    with rasterio.open(tif_path) as src:
        out_img, out_tr = mask(src, [aoi_geom], crop=True, filled=False)
        band = out_img[0]

        arr = band.astype("float64")
        arr[band.mask] = np.nan

        if src.nodata is not None:
            arr[arr == src.nodata] = np.nan

        if is_kelvin:
            arr -= 273.15

        if np.all(~np.isfinite(arr)):
            return np.nan

        rows = np.arange(arr.shape[0])
        ys = out_tr.f + (rows + 0.5) * out_tr.e
        w = np.cos(np.deg2rad(ys))[:, None]

        den = np.nansum(w * np.isfinite(arr))
        return np.nansum(arr * w) / den if den != 0 else np.nan

# =============================
# 4) Build AOI mean dataframe
# =============================
rows = []
for y, m, fp in pairs:
    mu = aoi_area_weighted_mean(fp, aoi_union, is_kelvin=OISST_IS_KELVIN)
    rows.append((y, m, pd.Timestamp(y, m, 1), mu))

df = pd.DataFrame(rows, columns=["year","month","date","sst"])
print(df.head(), "\nRows:", len(df))

# =============================
# 5) Monthly trend stats (outlier-removed)
# =============================
def month_trend_stats(df, mnum):
    d = df[df.month == mnum].sort_values("date")
    y = d.sst.values.astype(float)
    x = (d.date - d.date.min()).dt.days.values / 365.25

    keep = np.isfinite(y) & (y >= SST_MIN) & (y <= SST_MAX)
    if keep.sum() < 24:
        return None, d, keep

    res = linregress(x[keep], y[keep])
    yhat = np.full_like(y, np.nan, dtype=float)
    yhat[keep] = res.intercept + res.slope * x[keep]

    return {
        "trend": res.slope * 10.0,   # °C/decade
        "r2": res.rvalue**2,
        "p": res.pvalue,
        "yhat": yhat,
        "n": int(keep.sum())
    }, d, keep

# =============================
# 6) Pixel-wise trend map per month (drop bad timesteps by AOI mean)
# =============================
def trend_map_for_month(month_num, pairs, aoi_geom, min_count=24):
    files_m = [(y,m,fp) for (y,m,fp) in pairs if m == month_num]
    files_m.sort()

    good = []
    for y, m, fp in files_m:
        mu = aoi_area_weighted_mean(fp, aoi_geom, is_kelvin=OISST_IS_KELVIN)
        if np.isfinite(mu) and (SST_MIN <= mu <= SST_MAX):
            good.append((y, m, fp))

    if len(good) < min_count:
        raise ValueError(f"{month_names[month_num]}: too few good timesteps ({len(good)}). Try widen SST_MIN/MAX.")

    dates = pd.to_datetime([f"{y}-{m:02d}-01" for (y,m,_) in good])
    x = (dates - dates.min()).days.values / 365.25
    x = x.astype("float64")

    stack = []
    x_keep = []
    ref_tr = None
    ref_shape = None

    for (y,m,fp), xv in zip(good, x):
        with rasterio.open(fp) as src:
            out_img, out_tr = mask(src, [aoi_geom], crop=True, filled=False)
            band = out_img[0]
            arr = band.astype("float64")
            arr[band.mask] = np.nan

            if src.nodata is not None:
                arr[arr == src.nodata] = np.nan
            if OISST_IS_KELVIN:
                arr -= 273.15

            if ref_shape is None:
                ref_shape = arr.shape
                ref_tr = out_tr

        if arr.shape != ref_shape:
            continue

        # sanity clip (optional)
        arr[(arr < -5) | (arr > 50)] = np.nan

        stack.append(arr)
        x_keep.append(xv)

    if len(stack) < min_count:
        raise ValueError(f"{month_names[month_num]}: stack too small after cleaning ({len(stack)}).")

    stack = np.stack(stack, axis=0)               # (T,H,W)
    x3 = np.array(x_keep)[:, None, None]          # (T,1,1)

    valid = np.isfinite(stack)
    cnt = valid.sum(axis=0).astype("float64")     # (H,W)

    # safe mean (avoid divide-by-zero)
    xm = np.full_like(cnt, np.nan)
    ym = np.full_like(cnt, np.nan)
    okc = cnt > 0
    xm[okc] = np.sum(np.where(valid, x3, 0.0), axis=0)[okc] / cnt[okc]
    ym[okc] = np.sum(np.where(valid, stack, 0.0), axis=0)[okc] / cnt[okc]

    dx = x3 - xm
    dy = stack - ym

    cov = np.sum(np.where(valid, dx*dy, 0.0), axis=0)
    var = np.sum(np.where(valid, dx*dx, 0.0), axis=0)

    slope = np.full_like(ym, np.nan)
    ok = (cnt >= min_count) & (var > 0)
    slope[ok] = cov[ok] / var[ok]

    trend_map = (slope * 10.0).astype("float32")  # °C/decade
    return trend_map, ref_tr

# =============================
# 7) Summary CSV
# =============================
summary = []
for m in range(1, 13):
    stats, _, keep = month_trend_stats(df, m)
    if stats:
        summary.append([month_names[m], stats["trend"], stats["r2"], format_p(stats["p"]), stats["n"]])
    else:
        summary.append([month_names[m], np.nan, np.nan, "", int(np.sum(keep))])

df_sum = pd.DataFrame(summary, columns=["month","trend_C_decade","R2","p_value","n"])
csv_path = os.path.join(OUT_DIR, "Monthly_Trend_Summary_BoB.csv")
df_sum.to_csv(csv_path, index=False)
print("✅ Saved CSV:", csv_path)

# =============================
# 8) ONE BIG PNG (12× line + map)
# =============================
png_path = os.path.join(OUT_DIR, "AllMonths_Line_and_TrendMap.png")

fig, axes = plt.subplots(12, 2, figsize=(14, 26))
plt.subplots_adjust(hspace=0.35, wspace=0.05)

last_im = None

for m in range(1, 13):
    axL, axM = axes[m-1]

    stats, dmon, keep = month_trend_stats(df, m)

    axL.plot(dmon.date, dmon.sst, lw=0.8)
    if stats:
        axL.plot(dmon.date, stats["yhat"], lw=1.2)

        txt = (f"{month_names[m]}\n"
               f"Trend: {stats['trend']:.2f} °C/decade\n"
               f"R²: {stats['r2']:.2f}\n"
               f"p: {format_p(stats['p'])}")
    else:
        txt = f"{month_names[m]}\nNo valid trend"

    axL.text(0.02, 0.95, txt, transform=axL.transAxes,
             va="top", fontsize=8,
             bbox=dict(facecolor="white", alpha=0.85))
    axL.set_ylim(SST_MIN, SST_MAX)
    axL.set_ylabel("SST (°C)")
    axL.grid(alpha=0.25)
    if m != 12:
        axL.set_xticklabels([])

    try:
        tmap, tr = trend_map_for_month(m, pairs, aoi_union, min_count=24)
        H, W = tmap.shape
        left, top = tr.c, tr.f
        right = left + W * tr.a
        bottom = top + H * tr.e

        last_im = axM.imshow(
            tmap, origin="upper",
            extent=[left, right, bottom, top],
            vmin=VMIN, vmax=VMAX,
            interpolation="nearest"
        )
        aoi.boundary.plot(ax=axM, color="black", lw=0.8)

    except Exception as e:
        axM.axis("off")
        axM.text(0.5, 0.5, f"Map failed", ha="center", va="center", fontsize=8)

if last_im is not None:
    cbar = fig.colorbar(last_im, ax=axes[:, 1], fraction=0.03, pad=0.01)
    cbar.set_label("°C/decade")

fig.savefig(png_path, dpi=300, bbox_inches="tight")
plt.close(fig)
print("✅ Saved PNG:", png_path)

# =============================
# 9) PDF report (PNG + table)
# =============================
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image as RLImage, Table, TableStyle
from reportlab.lib.pagesizes import landscape, A4
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors

pdf_path = os.path.join(OUT_DIR, "OISST_BoB_Monthly_Report.pdf")
doc = SimpleDocTemplate(pdf_path, pagesize=landscape(A4))
styles = getSampleStyleSheet()

story = [
    Paragraph("OISST Bay of Bengal Monthly SST Trends (1982–2025)", styles["Title"]),
    Spacer(1, 8),
    RLImage(png_path, width=650, height=430),
    Spacer(1, 10)
]

tbl = Table([df_sum.columns.tolist()] + df_sum.values.tolist(), repeatRows=1)
tbl.setStyle(TableStyle([
    ("GRID", (0,0), (-1,-1), 0.4, colors.grey),
    ("BACKGROUND", (0,0), (-1,0), colors.lightgrey),
    ("FONTSIZE", (0,0), (-1,-1), 8),
]))
story.append(tbl)
doc.build(story)
print("✅ Saved PDF:", pdf_path)

# =============================
# 10) Download outputs
# =============================
files.download(png_path)
files.download(csv_path)
files.download(pdf_path)


⬆️ Upload AOI shapefile ZIP (contains .shp/.shx/.dbf/.prj)


Saving Bay of Bengal.zip to Bay of Bengal (3).zip
Uploaded: Bay of Bengal (3).zip
Found shp: ['/content/aoi_shape/Bay of Bengal/iho.shp']


/tmp/ipython-input-1513095621.py:76: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  aoi_union = aoi.unary_union


✅ AOI loaded
Total tif found: 524
Indexed (year,month,tif): 523
   year  month       date        sst
0  1982      1 1982-01-01  26.448879
1  1982      2 1982-02-01  26.940683
2  1982      3 1982-03-01  27.890862
3  1982      4 1982-04-01  29.087626
4  1982      5 1982-05-01  29.560397 
Rows: 523
✅ Saved CSV: /content/oisst_trend_outputs/Monthly_Trend_Summary_BoB.csv
✅ Saved PNG: /content/oisst_trend_outputs/AllMonths_Line_and_TrendMap.png
✅ Saved PDF: /content/oisst_trend_outputs/OISST_BoB_Monthly_Report.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
# --- Install (only once) ---
!pip -q install xlsxwriter

import os
import numpy as np
import pandas as pd
import rasterio
from google.colab import files

# =============================
# OUTPUT FOLDERS
# =============================
OUT_TIF_DIR = os.path.join(OUT_DIR, "Monthly_Trend_GeoTIFFs")
os.makedirs(OUT_TIF_DIR, exist_ok=True)

excel_path = os.path.join(OUT_DIR, "OISST_BoB_All_Data.xlsx")

# =============================
# Helper: save GeoTIFF
# =============================
def save_geotiff(arr, transform, out_fp, crs="EPSG:4326"):
    H, W = arr.shape
    profile = {
        "driver": "GTiff",
        "height": H,
        "width": W,
        "count": 1,
        "dtype": "float32",
        "crs": crs,
        "transform": transform,
        "nodata": -9999.0,
        "compress": "lzw"
    }
    arr2 = np.where(np.isfinite(arr), arr, profile["nodata"]).astype("float32")
    with rasterio.open(out_fp, "w", **profile) as dst:
        dst.write(arr2, 1)

# =============================
# 1) EXPORT 12 MONTHLY TREND GeoTIFFs
# =============================
print("🔹 Exporting monthly trend GeoTIFFs...")

trend_maps = {}  # keep for stats if needed later

for m in range(1, 13):
    # Use your clean trend-map function
    tmap, tr = trend_map_for_month(
        m,
        pairs,
        aoi_union,
        min_count=24
    )

    out_tif = os.path.join(
        OUT_TIF_DIR,
        f"OISST_BoB_Trend_{month_names[m]}_C_per_decade.tif"
    )

    save_geotiff(tmap, tr, out_tif)
    trend_maps[m] = tmap

    print("Saved:", out_tif)

# =============================
# 2) BUILD EXCEL WITH MULTIPLE SHEETS
# =============================
print("\n🔹 Writing Excel workbook with multiple sheets...")

with pd.ExcelWriter(excel_path, engine="xlsxwriter") as writer:

    # ---- Sheet 1: Full AOI monthly mean time series ----
    df_sorted = df.sort_values("date")
    df_sorted.to_excel(
        writer,
        sheet_name="AOI_Monthly_Mean_TimeSeries",
        index=False
    )

    # ---- Sheet 2: Monthly trend summary ----
    summary_rows = []
    for m in range(1, 13):
        stats, _, keep = month_trend_stats(df, m)
        if stats:
            summary_rows.append([
                month_names[m],
                stats["trend"],
                stats["r2"],
                stats["p"],
                stats["n"]
            ])
        else:
            summary_rows.append([
                month_names[m],
                np.nan, np.nan, np.nan, int(np.sum(keep))
            ])

    df_summary = pd.DataFrame(
        summary_rows,
        columns=["Month", "Trend_C_per_decade", "R2", "p_value", "N_points"]
    )

    df_summary.to_excel(
        writer,
        sheet_name="Monthly_Trend_Summary",
        index=False
    )

    # ---- Sheet 3–14: Month-wise AOI mean SST ----
    for m in range(1, 13):
        dmon = df[df["month"] == m].sort_values("date")
        dmon_out = dmon[["year", "month", "date", "sst"]].copy()

        dmon_out.to_excel(
            writer,
            sheet_name=month_names[m],
            index=False
        )

print("✅ Excel saved:", excel_path)

# =============================
# 3) DOWNLOAD OUTPUTS
# =============================
files.download(excel_path)

print("\n📦 GeoTIFFs are in folder:", OUT_TIF_DIR)
print("👉 You can zip & download them if you want.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 3.9 MB/s eta 0:00:00
🔹 Exporting monthly trend GeoTIFFs...
Saved: /content/oisst_trend_outputs/Monthly_Trend_GeoTIFFs/OISST_BoB_Trend_January_C_per_decade.tif
Saved: /content/oisst_trend_outputs/Monthly_Trend_GeoTIFFs/OISST_BoB_Trend_February_C_per_decade.tif
Saved: /content/oisst_trend_outputs/Monthly_Trend_GeoTIFFs/OISST_BoB_Trend_March_C_per_decade.tif
Saved: /content/oisst_trend_outputs/Monthly_Trend_GeoTIFFs/OISST_BoB_Trend_April_C_per_decade.tif
Saved: /content/oisst_trend_outputs/Monthly_Trend_GeoTIFFs/OISST_BoB_Trend_May_C_per_decade.tif
Saved: /content/oisst_trend_outputs/Monthly_Trend_GeoTIFFs/OISST_BoB_Trend_June_C_per_decade.tif
Saved: /content/oisst_trend_outputs/Monthly_Trend_GeoTIFFs/OISST_BoB_Trend_July_C_per_decade.tif
Saved: /content/oisst_trend_outputs/Monthly_Trend_GeoTIFFs/OISST_BoB_Trend_August_C_per_decade.tif
Saved: /content/oisst_trend_outputs/Monthly_Trend_GeoTIFFs/OISST_BoB_Trend_September_C_per_decad

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


📦 GeoTIFFs are in folder: /content/oisst_trend_outputs/Monthly_Trend_GeoTIFFs
👉 You can zip & download them if you want.


In [12]:
import os, glob, zipfile
from google.colab import files

# paths (আপনার আগের কোডে যেগুলো আছে)
OUT_TIF_DIR = os.path.join(OUT_DIR, "Monthly_Trend_GeoTIFFs")
excel_path = os.path.join(OUT_DIR, "OISST_BoB_All_Data.xlsx")

zip_path = os.path.join(OUT_DIR, "OISST_BoB_Outputs.zip")

# collect files to zip
tifs = sorted(glob.glob(os.path.join(OUT_TIF_DIR, "*.tif")))
to_zip = []

if os.path.exists(excel_path):
    to_zip.append(excel_path)
else:
    print("⚠️ Excel not found:", excel_path)

if len(tifs) > 0:
    to_zip.extend(tifs)
else:
    print("⚠️ No tif found in:", OUT_TIF_DIR)

# create zip
with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as z:
    for fp in to_zip:
        # keep clean names inside zip
        arcname = os.path.join("Monthly_Trend_GeoTIFFs", os.path.basename(fp)) if fp.endswith(".tif") else os.path.basename(fp)
        z.write(fp, arcname=arcname)

print("✅ ZIP created:", zip_path)
files.download(zip_path)


✅ ZIP created: /content/oisst_trend_outputs/OISST_BoB_Outputs.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files

# --- p format: no scientific, and NO significant text ---
def format_p(p):
    if p is None or (not np.isfinite(p)):
        return "NA"
    return f"{p:.6f}".rstrip("0").rstrip(".")

# ✅ choose which map-function you have
# If you have trend_map_for_month_clean() use that; else use trend_map_for_month()
MAP_FUNC = None
if "trend_map_for_month_clean" in globals():
    MAP_FUNC = trend_map_for_month_clean
elif "trend_map_for_month" in globals():
    MAP_FUNC = trend_map_for_month
else:
    raise NameError("trend_map_for_month_clean or trend_map_for_month not found.")

# Output
OUT_DIR = "/content/oisst_trend_outputs"
os.makedirs(OUT_DIR, exist_ok=True)
png_path = os.path.join(OUT_DIR, "AllMonths_Line_Box_Map.png")

# --- 12 rows × 3 cols (line | empty box | map) ---
fig, axes = plt.subplots(
    12, 3,
    figsize=(14, 24),
    gridspec_kw={"width_ratios":[2.2, 0.75, 1.05]}  # line big, box medium, map small
)

# gap কমানোর জন্য
plt.subplots_adjust(hspace=0.42, wspace=0.02)

last_im = None

for mnum in range(1, 13):
    axL = axes[mnum-1, 0]   # line
    axB = axes[mnum-1, 1]   # empty box
    axM = axes[mnum-1, 2]   # map

    # -----------------------------
    # (A) LINE + TREND (your outlier-filtered stats function)
    # -----------------------------
    if "month_trend_stats_from_df" in globals():
        stats, dmon, keep = month_trend_stats_from_df(df, mnum)
        yhat = stats["yhat"] if stats is not None else None
        trend = stats["trend_decade"] if stats is not None else np.nan
        r2 = stats["r2"] if stats is not None else np.nan
        p = stats["p"] if stats is not None else np.nan
    else:
        # fallback (simple)
        dmon = df[df["month"]==mnum].sort_values("date").copy()
        keep = np.isfinite(dmon["sst"].values)
        stats = None
        yhat = None
        trend = r2 = p = np.nan

    axL.plot(dmon["date"], dmon["sst"], lw=0.9)

    # trend line
    if yhat is not None:
        axL.plot(dmon["date"], yhat, lw=1.2)

    # outliers highlight (optional)
    if keep is not None:
        out_mask = np.isfinite(dmon["sst"].values) & (~keep)
        if np.any(out_mask):
            axL.scatter(dmon["date"].values[out_mask], dmon["sst"].values[out_mask], s=10)

    # compact title (NO significant text)
    if np.isfinite(trend):
        axL.set_title(
            f"{month_names[mnum]}  trend={trend:.2f} °C/decade, R²={r2:.2f}, p={format_p(p)}",
            fontsize=9
        )
    else:
        axL.set_title(f"{month_names[mnum]}  (trend NA)", fontsize=9)

    axL.set_ylabel("SST (°C)")
    axL.grid(alpha=0.25)

    # y-range fixed (24–32)
    if "SST_MIN" in globals() and "SST_MAX" in globals():
        axL.set_ylim(SST_MIN, SST_MAX)

    if mnum != 12:
        axL.set_xticklabels([])

    # -----------------------------
    # (B) EMPTY BOX (placeholder) — your provided blank box
    # -----------------------------
    axB.set_xticks([])
    axB.set_yticks([])
    # show only border like a box
    for sp in axB.spines.values():
        sp.set_visible(True)
        sp.set_linewidth(1.2)
    axB.set_facecolor("white")

    # -----------------------------
    # (C) MAP (trend map)
    # -----------------------------
    try:
        # for clean function signature differences
        if MAP_FUNC.__name__ == "trend_map_for_month_clean":
            tmap, tr = MAP_FUNC(mnum, pairs, aoi_union, is_kelvin=OISST_IS_KELVIN, min_count=24)
        else:
            tmap, tr = MAP_FUNC(mnum, pairs, aoi_union, is_kelvin=OISST_IS_KELVIN, min_count=24)

        H, W = tmap.shape
        left, top = tr.c, tr.f
        right = left + W * tr.a
        bottom = top + H * tr.e

        last_im = axM.imshow(
            tmap,
            origin="upper",
            interpolation="nearest",
            extent=[left, right, bottom, top],
            vmin=VMIN if "VMIN" in globals() else None,
            vmax=VMAX if "VMAX" in globals() else None,
        )

        # boundary overlay
        if "aoi" in globals():
            aoi.boundary.plot(ax=axM, color="black", linewidth=0.8)

        axM.set_title(f"{month_names[mnum]} map", fontsize=9)
        axM.tick_params(labelsize=7)

    except Exception as e:
        # map fail হলে clean way
        axM.axis("off")
        axM.text(0.5, 0.5, "Map failed", ha="center", va="center", fontsize=8)

# -----------------------------
# One colorbar for maps (right column)
# -----------------------------
if last_im is not None:
    cbar = fig.colorbar(last_im, ax=axes[:, 2], fraction=0.028, pad=0.01)
    cbar.set_label("°C/decade")

# tight save (close crop)
fig.savefig(png_path, dpi=300, bbox_inches="tight", pad_inches=0.03)
plt.close(fig)

print("✅ Saved:", png_path)
files.download(png_path)


In [ ]:
# --- Install (only once) ---
!pip -q install scipy

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
from google.colab import files

# Helper: pretty p-value without scientific notation (copied from ZYCD5olwXniq)
def format_p(p):
    if not np.isfinite(p):
        return "NA"
    if p < 0.00000001:
        return f"{p:.9f}"
    elif p < 0.001:
        return f"{p:.6f}"
    return f"{p:.3f}"


# 1) Trend stats for one month (slope/decade, R2, p)
def month_trend_stats(df, month_num, y_col="sst"):
    d = df[df["month"] == month_num].sort_values("date").copy()
    y = d[y_col].astype(float).values
    x = (d["date"] - d["date"].min()).dt.days.values / 365.25  # years since start

    m = np.isfinite(x) & np.isfinite(y)
    if m.sum() < 24:
        return None, d  # not enough data

    res = linregress(x[m], y[m])  # y = intercept + slope*x
    slope_decade = res.slope * 10.0
    r2 = res.rvalue**2
    p = res.pvalue

    signif = "significant" if p < 0.05 else "not significant"
    return {
        "slope_decade": slope_decade,
        "r2": r2,
        "p": p,
        "signif": signif,
        "intercept": res.intercept,
        "slope_year": res.slope,
        "x": x,
        "y": y,
        "mask": m
    }, d


# 2) One PNG with all months: 12 rows × 2 cols (line + map)
def make_all_months_png(
    df, pairs, aoi_union, trend_map_for_month, month_names,
    out_png="/content/oisst_trend_outputs/AllMonths_Line_and_TrendMap.png",
    vmin=-1.0, vmax=1.0
):
    fig, axes = plt.subplots(12, 2, figsize=(16, 30))  # big enough to read
    plt.subplots_adjust(hspace=0.55, wspace=0.0005)

    for mnum in range(1, 13):
        ax_line = axes[mnum-1, 0]
        ax_map  = axes[mnum-1, 1]

        # --- Line + trend ---
        stats, dmon = month_trend_stats(df, mnum, y_col="sst")

        ax_line.plot(dmon["date"], dmon["sst"], lw=1.0)

        if stats is not None:
            # trend line across full x
            x = stats["x"]
            yhat = stats["intercept"] + stats["slope_year"] * x
            ax_line.plot(dmon["date"], yhat, lw=1.2)

            # title text with stats, using format_p
            title = (f"{month_names[mnum]} trend = {stats['slope_decade']:.2f} °C/decade, "
                     f"R²={stats['r2']:.2f}, p={format_p(stats['p'])} ({stats['signif']})")
        else:
            title = f"{month_names[mnum]} (not enough valid data for trend)"

        ax_line.set_title(title, fontsize=9)
        ax_line.set_ylabel("SST (°C)")
        ax_line.grid(True, alpha=0.3)

        # --- Map (trend map for this month) ---
        try:
            tmap, t_transform = trend_map_for_month(
                mnum, pairs, aoi_union, is_kelvin=OISST_IS_KELVIN, min_count=24
            )

            H, W = tmap.shape
            left = t_transform.c
            top  = t_transform.f
            right = left + W * t_transform.a
            bottom = top + H * t_transform.e

            im = ax_map.imshow(
                tmap, origin="upper", interpolation="nearest",
                extent=[left, right, bottom, top],
                vmin=vmin, vmax=vmax
            )
            # AOI boundary overlay
            try:
                aoi.boundary.plot(ax=ax_map, color="black", linewidth=0.8)
            except:
                pass

            ax_map.set_title(f"{month_names[mnum]} trend map (°C/decade)", fontsize=9)
            ax_map.set_xlabel("Lon")
            ax_map.set_ylabel("Lat")

        except Exception as e:
            ax_map.text(0.5, 0.5, f"Map failed:\n{e}", ha="center", va="center")
            ax_map.set_axis_off()

    # one shared colorbar for all maps
    cbar = fig.colorbar(im, ax=axes[:, 1], fraction=0.02, pad=0.01)
    cbar.set_label("°C/decade")

    os.makedirs(os.path.dirname(out_png), exist_ok=True)
    fig.savefig(out_png, dpi=300, bbox_inches="tight")
    plt.close(fig)

    print("✅ Saved PNG:", out_png)
    return out_png


# 3) Make PNG + download
png_path = make_all_months_png(
    df=df,
    pairs=pairs,
    aoi_union=aoi_union,
    trend_map_for_month=trend_map_for_month,
    month_names=month_names,
    out_png="/content/oisst_trend_outputs/AllMonths_Line_and_TrendMap.png",
    vmin=-1.0, vmax=1.0
)

files.download(png_path)


In [ ]:
!pip -q install scipy

import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
from google.colab import files

# Helper: pretty p-value without scientific notation (copied from ZYCD5olwXniq)
def format_p(p):
    if not np.isfinite(p):
        return "NA"
    if p < 0.00000001:
        return f"{p:.9f}"
    elif p < 0.001:
        return f"{p:.6f}"
    return f"{p:.3f}"


def month_trend_stats(df, month_num):
    d = df[df["month"] == month_num].sort_values("date").copy()
    y = d["sst"].astype(float).values
    x = (d["date"] - d["date"].min()).dt.days.values / 365.25  # years since month-start

    m = np.isfinite(x) & np.isfinite(y)
    if m.sum() < 24:
        return d, None

    res = linregress(x[m], y[m])
    slope_decade = res.slope * 10.0
    r2 = res.rvalue**2
    p = res.pvalue
    signif = "significant" if p < 0.05 else "not significant"

    stats = {
        "slope_decade": slope_decade,
        "r2": r2,
        "p": p,
        "signif": signif,
        "slope_year": res.slope,
        "intercept": res.intercept,
        "mask": m,
        "x": x
    }
    return d, stats


def make_12_months_line_png(
    df,
    month_names,
    out_png="/content/oisst_trend_outputs/AllMonths_LineGraphs_2x6.png",
    ylim=(24, 32)  # আপনার চাইলে বদলাতে পারবেন
):
    fig, axes = plt.subplots(6, 2, figsize=(14, 18), sharey=True)
    axes = axes.flatten()

    for idx, mnum in enumerate(range(1, 13)):
        ax = axes[idx]
        d, stats = month_trend_stats(df, mnum)

        # line
        ax.plot(d["date"], d["sst"], lw=1.2)

        # trend line
        if stats is not None:
            x = stats["x"]
            yhat = stats["intercept"] + stats["slope_year"] * x
            ax.plot(d["date"], yhat, lw=1.2)

            txt = (f"{month_names[mnum]}\n"
                   f"trend = {stats['slope_decade']:.2f} °C/decade\n"
                   f"R² = {stats['r2']:.2f}\n"
                   f"p = {format_p(stats['p'])} ({stats['signif']})") # Use format_p here
        else:
            txt = f"{month_names[mnum]}\nNot enough valid data"

        # stats box inside plot
        ax.text(
            0.02, 0.98, txt,
            transform=ax.transAxes,
            va="top", ha="left",
            fontsize=9,
            bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.85, edgecolor="black")
        )

        ax.grid(True, alpha=0.25)
        ax.set_title("")  # title box already has month name

        if ylim is not None:
            ax.set_ylim(*ylim)

        # x label only bottom row
        if idx >= 10:
            ax.set_xlabel("Year")
        if idx % 2 == 0:
            ax.set_ylabel("SST (°C)")

    os.makedirs(os.path.dirname(out_png), exist_ok=True)
    plt.tight_layout()
    fig.savefig(out_png, dpi=300, bbox_inches="tight")
    plt.close(fig)
    print("✅ Saved:", out_png)
    return out_png


png_path = make_12_months_line_png(
    df=df,
    month_names=month_names,
    out_png="/content/oisst_trend_outputs/AllMonths_LineGraphs_2x6.png",
    ylim=(None)   # চাইলে None করে দিতে পারেন auto-scale এর জন্য
)

files.download(png_path)


In [ ]:
# --- Install (only once) ---
!pip -q install scipy

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
from google.colab import files

# 1) Trend stats for one month (slope/decade, R2, p)
def month_trend_stats(df, month_num, y_col="sst"):
    d = df[df["month"] == month_num].sort_values("date").copy()
    y = d[y_col].astype(float).values
    x = (d["date"] - d["date"].min()).dt.days.values / 365.25  # years since start

    m = np.isfinite(x) & np.isfinite(y)
    if m.sum() < 24:
        return None, d  # not enough data

    res = linregress(x[m], y[m])  # y = intercept + slope*x
    slope_decade = res.slope * 10.0
    r2 = res.rvalue**2
    p = res.pvalue

    signif = "significant" if p < 0.05 else "not significant"
    return {
        "slope_decade": slope_decade,
        "r2": r2,
        "p": p,
        "signif": signif,
        "intercept": res.intercept,
        "slope_year": res.slope,
        "x": x,
        "y": y,
        "mask": m
    }, d


# 2) One PNG with all months: 12 rows × 2 cols (line + map)
def make_all_months_png(
    df, pairs, aoi_union, trend_map_for_month, month_names,
    out_png="/content/oisst_trend_outputs/AllMonths_Line_and_TrendMap.png",
    vmin=-1.0, vmax=1.0
):
    fig, axes = plt.subplots(12, 2, figsize=(16, 30))  # big enough to read
    plt.subplots_adjust(hspace=0.55, wspace=0.20)

    for mnum in range(1, 13):
        ax_line = axes[mnum-1, 0]
        ax_map  = axes[mnum-1, 1]

        # --- Line + trend ---
        stats, dmon = month_trend_stats(df, mnum, y_col="sst")

        ax_line.plot(dmon["date"], dmon["sst"], lw=1.0)

        if stats is not None:
            # trend line across full x
            x = stats["x"]
            yhat = stats["intercept"] + stats["slope_year"] * x
            ax_line.plot(dmon["date"], yhat, lw=1.2)

            # title text with stats
            title = (f"{month_names[mnum]} trend = {stats['slope_decade']:.2f} °C/decade, "
                     f"R²={stats['r2']:.2f}, p={stats['p']:.3g} ({stats['signif']})")
        else:
            title = f"{month_names[mnum]} (not enough valid data for trend)"

        ax_line.set_title(title, fontsize=9)
        ax_line.set_ylabel("SST (°C)")
        ax_line.grid(True, alpha=0.3)

        # --- Map (trend map for this month) ---
        try:
            tmap, t_transform = trend_map_for_month(
                mnum, pairs, aoi_union, is_kelvin=OISST_IS_KELVIN, min_count=24
            )

            H, W = tmap.shape
            left = t_transform.c
            top  = t_transform.f
            right = left + W * t_transform.a
            bottom = top + H * t_transform.e

            im = ax_map.imshow(
                tmap, origin="upper", interpolation="nearest",
                extent=[left, right, bottom, top],
                vmin=vmin, vmax=vmax
            )
            # AOI boundary overlay
            try:
                aoi.boundary.plot(ax=ax_map, color="black", linewidth=0.8)
            except:
                pass

            ax_map.set_title(f"{month_names[mnum]} trend map (°C/decade)", fontsize=9)
            ax_map.set_xlabel("Lon")
            ax_map.set_ylabel("Lat")

        except Exception as e:
            ax_map.text(0.5, 0.5, f"Map failed:\n{e}", ha="center", va="center")
            ax_map.set_axis_off()

    # one shared colorbar for all maps
    cbar = fig.colorbar(im, ax=axes[:, 1], fraction=0.02, pad=0.01)
    cbar.set_label("°C/decade")

    os.makedirs(os.path.dirname(out_png), exist_ok=True)
    fig.savefig(out_png, dpi=300, bbox_inches="tight")
    plt.close(fig)

    print("✅ Saved PNG:", out_png)
    return out_png


# 3) Make PNG + download
png_path = make_all_months_png(
    df=df,
    pairs=pairs,
    aoi_union=aoi_union,
    trend_map_for_month=trend_map_for_month,
    month_names=month_names,
    out_png="/content/oisst_trend_outputs/AllMonths_Line_and_TrendMap.png",
    vmin=-1.0, vmax=1.0
)

files.download(png_path)


In [ ]:
# ===== ADD-ON CELL: 12 months line+trend + 12 months trend maps in ONE PNG + download =====
!pip -q install scipy

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress
import rasterio
from rasterio.mask import mask
from google.colab import files

# ---- Safety checks ----
needed = ["df", "pairs", "aoi_union", "month_names"]
for v in needed:
    if v not in globals():
        raise NameError(f"Missing variable: {v}. Please run the previous cells that create {v}.")

# ---- Define trend_map_for_month if not defined ----
if "trend_map_for_month" not in globals():

    def trend_map_for_month(selected_month_num, pairs, aoi_geom, is_kelvin=False, min_count=24):
        # collect files for that month
        month_files = [(y,m,fp) for (y,m,fp) in pairs if m == selected_month_num]
        month_files.sort()

        if len(month_files) < min_count:
            raise ValueError(f"Too few files for this month: {len(month_files)}")

        dates = pd.to_datetime([f"{y}-{m:02d}-01" for (y,m,fp) in month_files])
        x = (dates - dates.min()).days.values / 365.25
        x = x.astype("float64")

        stack = []
        x_keep = []
        ref_transform = None
        ref_shape = None

        for (y,m,fp), xval in zip(month_files, x):
            with rasterio.open(fp) as src:
                # filled=False -> masked array safe
                out_img, out_transform = mask(src, [aoi_geom], crop=True, filled=False)
                band = out_img[0]
                arr = band.astype("float64")
                arr[band.mask] = np.nan

                if src.nodata is not None:
                    arr[arr == src.nodata] = np.nan
                if is_kelvin:
                    arr = arr - 273.15

                if ref_shape is None:
                    ref_shape = arr.shape
                    ref_transform = out_transform

            # skip shape mismatch
            if arr.shape != ref_shape:
                continue

            stack.append(arr)
            x_keep.append(xval)

        if len(stack) < min_count:
            raise ValueError(f"Too few valid rasters kept: {len(stack)} (<{min_count}).")

        stack = np.stack(stack, axis=0)  # (T,H,W)
        x_keep = np.array(x_keep, dtype="float64")

        T, H, W = stack.shape
        valid = np.isfinite(stack)
        count = valid.sum(axis=0)
        enough = count >= min_count

        x3 = x_keep[:, None, None]

        # safe mean via sum/count (no warnings)
        x_sum = np.sum(np.where(valid, x3, 0.0), axis=0)
        y_sum = np.sum(np.where(valid, stack, 0.0), axis=0)

        x_mean = np.full((H,W), np.nan, dtype="float64")
        y_mean = np.full((H,W), np.nan, dtype="float64")
        anym = count > 0
        x_mean[anym] = x_sum[anym] / count[anym]
        y_mean[anym] = y_sum[anym] / count[anym]

        dx = x3 - x_mean
        dy = stack - y_mean

        cov = np.sum(np.where(valid, dx*dy, 0.0), axis=0)
        var = np.sum(np.where(valid, dx*dx, 0.0), axis=0)

        slope_per_year = np.full((H,W), np.nan, dtype="float64")
        ok = enough & (var > 0)
        slope_per_year[ok] = cov[ok] / var[ok]

        trend_map = (slope_per_year * 10.0).astype("float32")  # °C/decade
        return trend_map, ref_transform


# ---- Trend stats for a month (slope/decade, R2, p, significant) ----
def month_trend_stats(df, month_num):
    d = df[df["month"] == month_num].sort_values("date").copy()
    y = d["sst"].astype(float).values
    x = (d["date"] - d["date"].min()).dt.days.values / 365.25

    m = np.isfinite(x) & np.isfinite(y)
    if m.sum() < 24:
        return None, d

    res = linregress(x[m], y[m])  # y = intercept + slope*x
    slope_decade = res.slope * 10.0
    r2 = res.rvalue**2
    p = res.pvalue
    signif = "significant" if p < 0.05 else "not significant"

    return {
        "slope_year": res.slope,
        "intercept": res.intercept,
        "slope_decade": slope_decade,
        "r2": r2,
        "p": p,
        "signif": signif,
    }, d


# ---- Make one PNG with 12 rows x 2 cols (line+trend, map) ----
def make_all_months_png(
    df, pairs, aoi_union, month_names,
    out_png="/content/oisst_trend_outputs/AllMonths_Line_and_TrendMap.png",
    vmin=-1.0, vmax=1.0
):
    os.makedirs(os.path.dirname(out_png), exist_ok=True)

    fig, axes = plt.subplots(12, 2, figsize=(16, 30))
    plt.subplots_adjust(hspace=0.60, wspace=0.20)

    last_im = None

    for mnum in range(1, 13):
        ax_line = axes[mnum-1, 0]
        ax_map  = axes[mnum-1, 1]

        # ---- line + trend ----
        stats, dmon = month_trend_stats(df, mnum)
        ax_line.plot(dmon["date"], dmon["sst"], lw=1.0)

        if stats is not None:
            x = (dmon["date"] - dmon["date"].min()).dt.days.values / 365.25
            yhat = stats["intercept"] + stats["slope_year"] * x
            ax_line.plot(dmon["date"], yhat, lw=1.2)

            ax_line.set_title(
                f"{month_names[mnum]} trend = {stats['slope_decade']:.2f} °C/decade, "
                f"R²={stats['r2']:.2f}, p={stats['p']:.3g} ({stats['signif']})",
                fontsize=9
            )
        else:
            ax_line.set_title(f"{month_names[mnum]} (not enough data for trend)", fontsize=9)

        ax_line.set_ylabel("SST (°C)")
        ax_line.grid(True, alpha=0.3)

        # ---- map ----
        try:
            tmap, t_transform = trend_map_for_month(
                mnum, pairs, aoi_union,
                is_kelvin=OISST_IS_KELVIN if "OISST_IS_KELVIN" in globals() else False,
                min_count=24
            )

            H, W = tmap.shape
            left = t_transform.c
            top  = t_transform.f
            right = left + W * t_transform.a
            bottom = top + H * t_transform.e

            last_im = ax_map.imshow(
                tmap, origin="upper", interpolation="nearest",
                extent=[left, right, bottom, top],
                vmin=vmin, vmax=vmax
            )

            # AOI boundary overlay (if aoi exists)
            if "aoi" in globals():
                try:
                    aoi.boundary.plot(ax=ax_map, color="black", linewidth=0.7)
                except:
                    pass

            ax_map.set_title(f"{month_names[mnum]} trend map (°C/decade)", fontsize=9)
            ax_map.set_xlabel("Lon")
            ax_map.set_ylabel("Lat")

        except Exception as e:
            ax_map.text(0.5, 0.5, f"Map failed:\n{str(e)[:120]}", ha="center", va="center", fontsize=8)
            ax_map.set_axis_off()

    # shared colorbar
    if last_im is not None:
        cbar = fig.colorbar(last_im, ax=axes[:, 1], fraction=0.02, pad=0.01)
        cbar.set_label("°C/decade")

    fig.savefig(out_png, dpi=300, bbox_inches="tight")
    plt.close(fig)

    print("✅ Saved PNG:", out_png)
    return out_png


# ---- Run + download ----
png_path = make_all_months_png(
    df=df,
    pairs=pairs,
    aoi_union=aoi_union,
    month_names=month_names,
    out_png="/content/oisst_trend_outputs/AllMonths_Line_and_TrendMap.png",
    vmin=-1.0, vmax=1.0
)

files.download(png_path)
